# 단어 임베딩 사용하기
> 단어와 벡터를 연관 짓는 강력하고 인기있는 방법은 **단어 임베딩**이라는 밀집 **단어 벡터**를 사용하는 것이다. 원-핫 인코딩으로 만든 벡터는 희소하고(대부분 0으로 채워짐) 고차원이다(어휘 단어에 있는 단어의 수와 차원이 같다!!!). 반면에 단어 임베딩은 저차원의 실수형 벡터이다. 원-핫 인코딩 보다 단어 임베딩이 더 많은 정보를 적은 차원에 저장한다.
-------------------
* 원-핫 인코딩
> ##### 희소
> ##### 고차원
> ##### 수동 인코딩
* 단어 임베딩
> ##### 밀집
> ##### 저차원
> ##### 데이터로부터 학습
--------------------
#### 단어 임베딩을 만드는 방법은 두가지 이다.
* (문서 분류나 감성 예측 같은) 관심 대상인 문제와 함께 단어 임베딩을 학습한다. 이런 경우에는 랜덤한 단어 벡터로 시작해서 신경망의 가중치를 학습하는 것과 같은 방식으로 단어 벡터를 학습한다.
* 풀려는 문제가 아니고 다른 머신 러닝 작업에서 미리 계산된 단어 임베딩을 로딩한다. 이를 **사전 훈련된 단어 임베딩(pretrained word embedding)**이라고 한다.
--------------------------
## Embedding 층을 사용하여 단어 임베딩 학습하기
> 단어와 밀집 벡터를 연관 짓는 가장 간단한 방법은 랜덤하게 벡터를 선택하는 것이다. 이 방식의 문제점은 임베딩 공간이 구조적이지 않다는 것이다. 예를 들어 accurate(정확한)와 exact(정확한) 단어는 대부분의 문장에서 비슷한 의미로 사용되지만 완전히 다른 임베딩을 가진다. 심층 신경망이 이런 임의의 구조적이지 않은 임베딩 공간을 이해하기는 어렵다!! 그러기에 추상적인 관계를 얻으려면 단어 사이에 있는 의미 관계를 반영해야 한다. 단어 임베딩은 언어를 기하학적 공간에 매핑하는 것이다. 예를 들어 잘 구축된 임베딩 공간에서는 동의어가 비슷한 단어 벡터로 임베딩 될 것 이다. 일반적으로 두 단어 벡터사이의 거리는 이 단어 사이의 의미 거리와 관계되어 있다. 거리 외에 임베딩 곤간의 특정 방향도 의미를 가질 수 있다.


### Embedding 층의 객체 생성하기

In [1]:
from keras.layers import Embedding

embedding_layer = Embedding(1000, 64)
# Embedding 층은 적어도 2개의 매개변수를 받는다. 가능한 토큰 개수(여기서는 1000으로 단어 인덱스 최댓값+1이다.)와 임베딩 차원(64)이다.


### Embedding 층
### 단어 인덱스 -> Embedding 층 -> 연관된 단어 벡터
------------------------------
#### Embedding 층은 크기가 (samples, sequence_length)인 2D정수 텐서를 입력으로 받는다. 각 샘플은 정수의 시퀀스이다. 배치에 있는 모든 시퀀스는 길이가 같아야 하므로 작은 길이의 시퀀스는 0으로 패딩되고 길이가 더 긴 시퀀스는 짤린다.
#### Embedding 층은 크기가 (samples, sequence_length, embedding_dimensionality) 인 3D 실수형 텐서를 반환한다. 이런 3D텐서는 RNN층이나 1D 합성곱 층에서 처리된다.
#### Embedding 층의 객체를 생성할 때 가중치(토큰 벡터를 위한 내부 딕셔너리)는 다른 층과 마찬가지로 랜덤하게 초기화 된다. 훈련하면서 이 단어 벡터는 역전파를 통해 점차 조정되어 이어지는 모델이 사용할 수 있도록 임베딩 공간을 구성한다.  훈련이 끝나면 임베딩 공간은 특정 문제에 특화한 문제를 많이 가지게 된다.
------------------------
#### IMDB 영화 리뷰 감성 예측 문제에 적용해 보자.
### Embedding 층에 사용할 IMDB 데이터 로드하기

In [2]:
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000 # 특성으로 사용할 단어의 수
maxlen = 20 # 사용할 텍스트의 길이 (가장 빈번한 max_features개의 단어만 사용한다.)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features) # 정수 리스트로 데이터 로드

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant t

In [5]:
x_test[0]

[1,
 591,
 202,
 14,
 31,
 6,
 717,
 10,
 10,
 2,
 2,
 5,
 4,
 360,
 7,
 4,
 177,
 5760,
 394,
 354,
 4,
 123,
 9,
 1035,
 1035,
 1035,
 10,
 10,
 13,
 92,
 124,
 89,
 488,
 7944,
 100,
 28,
 1668,
 14,
 31,
 23,
 27,
 7479,
 29,
 220,
 468,
 8,
 124,
 14,
 286,
 170,
 8,
 157,
 46,
 5,
 27,
 239,
 16,
 179,
 2,
 38,
 32,
 25,
 7944,
 451,
 202,
 14,
 6,
 717]

In [6]:
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)
# 리스트를 (samples, maxlen) 크기의 2D정수 텐서로 변환한다.
# pad_sequences() 함수는 패딩을 넣을 위치를 지정하는 padding 매개변수가 있다. 기본값 pre는 시퀀스 왼쪽에 패딩이 추가되고 post는 오른쪽에 패딩이 추가된다.

### IMDB데이터에 Embedding 층과 분류기 사용하기

In [8]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen)) 
# 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해
# embedding 층에서 input_length의 크기를 지정한다.

model.add(Flatten()) # 3d 임베딩 텐서를 (samples, maxlen * 8) 크기의 2D텐서로 펼친다.

model.add(Dense(1, activation='sigmoid')) # 분류기를 추가한다.
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                   epochs=10,
                   batch_size = 32, 
                   validation_split=0.2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 1s 1ms/step - loss: 0.6880 - acc: 0.5472 - val_loss: 0.6309 - val_acc: 0.6898
Epoch 2/10
625/625 [==============================] - 1s 806us/step - loss: 0.5800 - acc: 0.7437 - val_loss: 0.5318 - val_acc: 0.7276
Epoch 3/10
625/625 [==============================] - 0s 762us/step - loss: 0.4752 - acc: 0.7833 - val_loss: 0.5033 

약 75%의 정확도가 나온다. 20개의 단어만 사용한 것 치고 꽤 좋은 결과이다. 하지만 단어 사이의 관계나 문장 구조를 고려하지 않아으므로 정확하지 않다. 각 시퀀스 전체를 고려한 특성을 학습하도록 임베딩 층 위에 순환 층이나 1D합성곱층을 추가하여야 한다.